# Análisis de Sentimientos a Nivel de Texto: Preprocesamiento de los Datos
---

> **Proyecto final de Asignatura Sistemas Computacionales <br>
Escuela de Ingeniería de Sistemas <br>
Universidad de Los Andes <br>
Autor: Jhonathan Abreu <br>**

---
<br>
La función de este notebook es describir las primeras etapas del procesamiento de la data que alimentará el modelo de clasificación para el análisis de sentimientos. En este caso, se tomarán los datos del [formulario](https://docs.google.com/forms/d/e/1FAIpQLSdugT3KgsfQEmvB0XVnWhP3Cd5t1fSSJ7mqBY2-I_IZyxBYew/viewform?usp=sf_link) de Google Forms utilizado para la recolección de datos. El resultado final será un archivo CSV con todas las frases y su respectivas etiquetas (positivo, negativo, neutral).

## Bibliotecas y módulos necesarios para el proyecto
---

Este y los demás notebooks están implementados para ser ejecutados con `Python 3`. Se deben instalar los siguientes módulos de para la ejecución de este y los demás notebooks correspondientes al presente proyecto:

*  Pandas (v0.22.0)
*  Numpy (v1.14.5)
*  NLTK (v3.2.5)
*  sklearn (scikit-learn, v0.19.1)
*  Keras (v2.1.6)
*  TensorFlow (opcional, si la instalación de Keras lo requiere, v1.9.0rc0)
*  matplotlib (v2.1.2)
*  seaborn (v0.7.1)

Por lo general, los módulos Pandas, Numpy, sklearn, matplotlib y seaborn son distribuídos con [Anaconda](https://anaconda.org/anaconda/python), el cual también provee una distribución del paquete `jupyter-lab`, el cual permite visualizar y editar notebooks de Python. Las versiones mostradas son las instaladas en Colaboratory, con un entorno de Python 3. Se recomienda utilizar Colaboratory o, en caso de usar un entorno local, instalar Anaconda y los paquetes que sean necesarios (principalmente TensorFlow, Keras y NLTK).

## Base de datos
---

Los datos a ser utilizados para el entrenamiento y prueba del clasificador se encuentran alojados en Google Drive y pueden descargarse en el siguiente enlace:

> [Datos de entrenamiento](https://drive.google.com/file/d/1CoaP5GcoVilWu1hYansyO06pvKCb1Ia8/view?usp=sharing)

## Configuración del sistemas de archivos de Google Drive
---

Este proyecto está diseñado para ser poder ser ejecutado en un entorno con Python 3 en Google Colaboratory, por lo cual, los datasets deben estar ubicados en un directorio de su Drive. A continuación, se muestra la configuración del sistema de archivos. Si desea utilizar Colaboratory, ejecute la siguiente celda de código para configurar el sistema de archivos de su Drive.

> **Nota 1**: durante la configuración, se le va a solicitar dos (2) veces abrir un enlace, inicicar sesión con su cuenta de Google y copiar y pegar un token de autenticación.

> **Nota 2**: específicamente, el arbol de directorios que se muestra en el bloque de declaración de constantes, debe replicarse para el correcto funcionamiento de estos notebooks.


In [1]:
# Instalar la biblioteca FUSE (Filesystem in Userspase) para manejar el sistema
# de archivos de Google Drive
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties \
    module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Importar bibliotecas necesarias para autenticación
from google.colab import auth
from oauth2client.client import GoogleCredentials
import getpass

# Generar los tokens de autorización para Colaboratory
auth.authenticate_user()

# Generar credenciales para la biblioteca FUSE
credentials = GoogleCredentials.get_application_default()

!google-drive-ocamlfuse -headless -id={credentials.client_id} \
    -secret={credentials.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={credentials.client_id} \
    -secret={credentials.client_secret}

# Montar sistema de archivos
!fusermount -u drive
!sshfs -u drive

# Crear un directorio y montar Google Drive usando ese directorio
!mkdir -p drive
!google-drive-ocamlfuse drive

# Prueba: mostrar contenido de un directorio en drive
print ('Archivos en Drive:')
!ls drive

gpg: keybox '/tmp/tmps9mbp_uy/pubring.gpg' created
gpg: /tmp/tmps9mbp_uy/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········
/bin/sh: 1: sshfs: not found
Archivos en Drive:
aplicacion.ipynb  mejormodelo.ipynb  modelos
datasets	  modelo.ipynb	     preprocesamiento.ipynb


## Declaración de constantes
---

Las siguientes constantes son necesarias para ubicar los datasets y los modelos. Modifique según sea necesario, si va a utilizar Colaboratory o un entorno local.

In [4]:
import os

# Directorios de los datasets

#   NOTA: cambiar PROJECT_DIR al directorio raíz del proyecto
#     Para drive, cambie la siguiente variable a la ruta del
#     proyecto en su Drive:
#PROJECT_DIR = 'drive/ULA/sistemascomputacionales/aplicacion'
#     Para entorno local, cambie la siguiente variable para apunta
#     a la ruta abosluta del proyecto en su disco:
PROJECT_DIR = ('/home/jhonathanabreu/Documentos/ula/sistemas_computacionales/'
               'Proyecto_SistemasComputacionales_AbreuJ/aplicacion')
DATASETS_DIR = os.path.join(PROJECT_DIR, 'datasets')
ORIGINAL_DATASETS_DIR = os.path.join(DATASETS_DIR, 'originales')
PROCESSED_DATASETS_DIR = os.path.join(DATASETS_DIR, 'procesados')
MODELS_DIR = os.path.join(PROJECT_DIR, 'modelos')

# Etiquetas de los sentimientos
POSITIVE_SENTIMENT_LABEL = 'positivo'
NEGATIVE_SENTIMENT_LABEL = 'negativo'
NEUTRAL_SENTIMENT_LABEL = 'neutral'

## Preprocesamiento de los datos
---

El procesamiento inicial de los datos es sencillo y consiste únicamente en:

1.   Obtener las oraciones del archivo CSV generado por el formulario
2.   Etiquetar cada oración según la columna a la que pertenece:
     *   Columna 4: oraciones positivas
     *   Columna 5:oraciones negativas
     *   Columna 6: oraciones neutrales
3.   Escribir las oraciones con sus etiquetas en un archivo CSV de salida.

<br>
### Preprocesamiento de los datos del formulario

Esta es la etapa final del preprocesamiento del formulario, en la cual se produce el archivo CSV con los datos preparados para ser alimentados al clasificador.

La siguiente función se encarga de realizar lo anterior mencionado:



In [5]:
import xml.etree.ElementTree as ET
import csv

#-------------------------------------------------------------------------------
# Función de carga de los datos del formulario de Google Forms.
#
# Parámetros:
#    inputFileName: ruta del archivo CSV del formulario.
#    outputFileName: ruta del archivo CSV para escribir las oraciones
#                    etiquetadas.
#
#-------------------------------------------------------------------------------
def getDataFromForm(inputFileName, outputFileName):
    # En las columnas 5, 6 y 7 se encuentran las frases positiva, negativa y
    # neutral, respectivamente.
    possitiveSentimentRow = 4
    negativeSentimentRow = 5
    neutralSentimentRow = 6
    
    sentences = {
        'sentence': [],
        'sentiment': []
    }
    
    with open(inputFileName, newline = '') as formDataFile:
        with open(outputFileName, 'w') as labeledDataFile:
            # Lector del archivo original
            csvReader = csv.reader(formDataFile)
            # Escritor del archivo de data etiquetada
            csvWriter = csv.writer(labeledDataFile)
            
            csvReader.__next__()  # Saltar el header
            
            for row in csvReader:
                # Se escriben las oraciones en las columnas 4, 5 y 6
                # con las etiquetas positivo, negativo y neutral,
                # respectivamente.
                csvWriter.writerow([row[possitiveSentimentRow],
                                    POSITIVE_SENTIMENT_LABEL])
                csvWriter.writerow([row[negativeSentimentRow],
                                    NEGATIVE_SENTIMENT_LABEL])
                csvWriter.writerow([row[neutralSentimentRow],
                                    NEUTRAL_SENTIMENT_LABEL])

formDataFile = os.path.join(ORIGINAL_DATASETS_DIR, 'dataset.csv')
labeledDataFile = os.path.join(PROCESSED_DATASETS_DIR, 'labeled_data.csv')

getDataFromForm(formDataFile, labeledDataFile)

!ls {PROCESSED_DATASETS_DIR}


labeled_data.csv
